#### Home work

Q1. What's the version.build_hash value?

In [1]:
!curl localhost:9200

{

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   539  100   539    0     0  80628      0 --:--:-- --:--:-- --:--:-- 89833



  "name" : "a06cc2832ec8",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "734zLd0SR6mB6waoH426GQ",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


This is the version.build_hash: 42f05b9372a9a4a470db3b52817899b99a76ee73

Getting the data

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

Q2. Indexing the data 

In [10]:
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

c:\Users\karth\Programming\LLM-ZoomCamp\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
es_client = Elasticsearch('http://localhost:9200')

In [8]:
index_settings = {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },
        "mappings": {
            "properties": {
                    "text": {"type": "text"},
                    "section": {"type": "text"},
                    "question": {"type": "text"},
                    "course": {"type": "keyword"} 
        }
     }
}


index_name = "faq-index"

es_client.indices.create(
    index=index_name,
    body=index_settings
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'faq-index'})

In [11]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:47<00:00, 19.81it/s]


Q3. Searching 

In [13]:
query = "How do execute a command on a Kubernetes pod?"

In [20]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
        }
    }
}

In [21]:
response = es_client.search(index=index_name,
                 body=search_query)

In [22]:
response 

ObjectApiResponse({'took': 11, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 739, 'relation': 'eq'}, 'max_score': 44.50556, 'hits': [{'_index': 'faq-index', '_id': 'W0sreJcB3ekf7TEoC6_W', '_score': 44.50556, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'faq-index', '_id': '6ksreJcB3ekf7TEoKK8K', '_score': 35.433445, '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke', 'section': '10. Kubernetes and TensorFlow Serving', 'question': 'Kubernetes-dashboard

In [ ]:
   response['hits']

{'total': {'value': 739, 'relation': 'eq'},
 'max_score': 44.50556,
 'hits': [{'_index': 'faq-index',
   '_id': 'W0sreJcB3ekf7TEoC6_W',
   '_score': 44.50556,
   '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
    'section': '5. Deploying Machine Learning Models',
    'question': 'How do I debug a docker container?',
    'course': 'machine-learning-zoomcamp'}},
  {'_index': 'faq-index',
   '_id': '6ksreJcB3ekf7TEoKK8K',
   '_score': 35.433445,
   '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke',
    'section': '10. Kubernetes and TensorFlow Serving',
    'question': 'Kubernetes-dashboard',
    'course': 'machine-learning-zoomcamp'}},
  {'_index': 'faq-index',
   '_id': 'e

Q4. Filtering

In [25]:
query = "How do copy a file to a Docker container?"

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [26]:
response = es_client.search(index=index_name,
                 body=search_query)

In [28]:
response['hits']['hits']

[{'_index': 'faq-index',
  '_id': 'W0sreJcB3ekf7TEoC6_W',
  '_score': 73.38676,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'faq-index',
  '_id': 'eksreJcB3ekf7TEoEa_k',
  '_score': 66.688705,
  '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory c

In [34]:
response['hits']['hits'][2]['_source']['question']

'How do I copy files from a different folder into docker container’s working directory?'

Q5. Buidling a prompt

In [46]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [42]:
retrieved_docs = []

response = es_client.search(index=index_name,
                 body=search_query)


for hit in response['hits']['hits']:
        retrieved_docs.append(hit['_source'])

In [43]:
retrieved_docs

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [55]:
context = "" 

for doc in retrieved_docs:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [56]:
context

'section: 5. Deploying Machine Learning Models\nquestion: How do I debug a docker container?\nanswer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nsection: 5. Deploying Machine Learning Models\nquestion: How do I copy files from my local machine to docker container?\nanswer: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani\n\nsection: 5. Deploying Machine Learning Models\nquestion: How do I cop

In [57]:
query = "How do copy a file to a Docker container?"

prompt = prompt_template.format(question=query, context=context).strip()

In [58]:
prompt

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\n\nQUESTION: How do copy a file to a Docker container?\n\nCONTEXT:\nsection: 5. Deploying Machine Learning Models\nquestion: How do I debug a docker container?\nanswer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nsection: 5. Deploying Machine Learning Models\nquestion: How do I copy files from my local machine to docker container?\nanswer: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you 

In [59]:
len(prompt)

2424

Q6. Tokens

In [60]:
import tiktoken

In [61]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [64]:
encodings = encoding.encode(prompt)

In [65]:
len(encodings)

538

In [66]:
encoding.decode_single_token_bytes(63842)

b"You're"

Q7. Generating the answer

In [73]:
from dotenv import load_dotenv

from langchain_ollama import ChatOllama
from langchain_mistralai.chat_models import ChatMistralAI

load_dotenv()

True

In [74]:
def get_llm(model_name):
    if model_name == "mistral":
        llm = ChatMistralAI()
    elif model_name == "llama":
        llm = ChatOllama(model="llama3.2-3b") 
    else:
        print("Invalid model name...")
    return llm

In [75]:
def get_llm_response(prompt, model_name="mistral"): 
    llm = get_llm(model_name=model_name)
    response = llm.invoke(prompt)
    return response.content

In [76]:
answer = get_llm_response(prompt)

In [77]:
answer

'To copy a file to a Docker container, you can use the `docker cp` command. The basic syntax is as follows:\n\n`docker cp /path/to/local/file_or_directory container_id:/path/in/container`\n\nThis command allows you to copy a file or directory from your local machine into a running Docker container. The `container_id` refers to the specific container where you want to copy the file, and the `/path/in/container` is the destination path within the container. (Hrithik Kumar Advani)'

In [78]:
len(answer)

477

Q8. Calculating the costs

In [80]:
# GPT-4.1 Pricing (per 1M tokens)
input_cost_per_1m = 2.00
output_cost_per_1m = 8.00

# Token usage per request
input_tokens_per_request = 538
output_tokens_per_request = 477
num_requests = 1000

# Total tokens
total_input_tokens = input_tokens_per_request * num_requests
total_output_tokens = output_tokens_per_request * num_requests

# Cost calculation
input_cost = (total_input_tokens / 1_000_000) * input_cost_per_1m
output_cost = (total_output_tokens / 1_000_000) * output_cost_per_1m
total_cost = input_cost + output_cost

# Output
print(f"Input cost: ${input_cost:.4f}")
print(f"Output cost: ${output_cost:.4f}")
print(f"Total cost for {num_requests} requests: ${total_cost:.4f}")

Input cost: $1.0760
Output cost: $3.8160
Total cost for 1000 requests: $4.8920
